# Fitting the ground blockage

Gamma is then used as a constant.

## 1. Load Data

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import math
import os
import glob
from scipy.optimize import least_squares
sns.set_style(style='white')
from matplotlib.backends.backend_pdf import PdfPages

# os.chdir("/mydir")
# %matplotlib inline
# %config InlineBackend.figure_format = 'svg'
label_size = 22
ticks_size = 22
legend_size = 20
marker_size = 12
linewidth_t = 4

In [5]:
ex0_0 = pd.read_csv('data/ground_data/ground0.csv')
ex0_1 = pd.read_csv('data/ground_data/ground1.csv')
ex0_2 = pd.read_csv('data/ground_data/ground2.csv')
# calculate ESP
ex0_0["ESP"] = ex0_0["RSSI"] + ex0_0["SNR"] - 10 * np.log10(1 + np.power(10, 0.1 * ex0_0["SNR"]))
ex0_1["ESP"] = ex0_1["RSSI"] + ex0_1["SNR"] - 10 * np.log10(1 + np.power(10, 0.1 * ex0_1["SNR"]))
ex0_2["ESP"] = ex0_2["RSSI"] + ex0_2["SNR"] - 10 * np.log10(1 + np.power(10, 0.1 * ex0_2["SNR"]))
# set node height
ex0_0["gz"] = 0.5
ex0_1["gz"] = 1
ex0_2["gz"] = 1.5
ex0_0["i"] = 0
ex0_1["i"] = 1
ex0_2["i"] = 2
ex0_0["j"] = 0
ex0_1["j"] = 0
ex0_2["j"] = 0
ex0_0["d"] = 100
ex0_1["d"] = math.pow(10000+0.5**2, 0.5)
ex0_2["d"] = math.pow(10000+1**2, 0.5)
# concatenate as the fitting data
testh = pd.concat([ex0_0, ex0_1, ex0_2])

In [6]:
# Struct of tree
class Tree:
    def __init__(self, loc_x, loc_y, mid_z, hori_r, vert_r, trunk_r):
        self.loc_x = loc_x
        self.loc_y = loc_y
        self.mid_z = mid_z
        self.hori_r = hori_r
        self.vert_r = vert_r
        self.trunk_h = mid_z - vert_r + 0.5
        self.trunk_r = trunk_r
        
    def CheckPtFoliage(self, x, y, z):
        if(((x-self.loc_x)*(x-self.loc_x))/(self.hori_r*self.hori_r) + ((y-self.loc_y)*(y-self.loc_y))/(self.hori_r*self.hori_r) + ((z-self.mid_z)*(z-self.mid_z))/(self.vert_r*self.vert_r) > 1): return False
        else: return True
        
    def CheckPtTrunk(self, x, y, z):
        if (z > self.trunk_h or z < 0): return False
        elif ((x-self.loc_x)*(x-self.loc_x) + (y-self.loc_y)*(y-self.loc_y) - self.trunk_r*self.trunk_r > 0): return False
        else: return True
        
def CheckPtGround(z):
    if(z > 0): return False
    else: return True


In [7]:
def SampPair(g_x, g_y, g_z, n_x, n_y, n_z, t_x, t_y, mid_z, topo, hori_r, vert_r, trunk_r, offset_x, offset_y, n, f, plane_gap, point_gap):
    sample_rec = []
    empties = [[6,1],[8,23],[10,23],[13,22],[17,21],[18,9],[20,15],[21,11],[22,22],[22,31],[23,11],[25,8],[27,16],[36,8],[37,8],[38,2],[38,16],[38,28],[40,8]]
    sample_rec = []
    distance_gn = math.pow(((g_x-n_x)*(g_x-n_x) + (g_y-n_y)*(g_y-n_y) + (g_z-n_z)*(g_z-n_z)), 0.5)
    distance_hori_gn = math.pow(((g_x-n_x)*(g_x-n_x) + (g_y-n_y)*(g_y-n_y)), 0.5)
    xy = math.pow(((g_x-n_x)*(g_x-n_x) + (g_y-n_y)*(g_y-n_y)), 0.5)
    xyz = math.pow(((g_x-n_x)*(g_x-n_x) + (g_y-n_y)*(g_y-n_y) + (g_z-n_z)*(g_z-n_z)), 0.5)
    xyoxyz = xy / xyz
    yoxy = (g_y-n_y) / xy
    xoxy = (g_x-n_x) / xy
    yoxyz = (g_y-n_y) / xyz
    xoxyz = (g_x-n_x) / xyz
    zoxyz = (g_z-n_z) / xyz
    # Given a pair, find the related trees
    trees = []
    print("For this pair, ", len(trees), " trees are suspicious.")     
    log_rates = []
    pl_log = 21.11
    pl_energy = 0
    area_ref = 4 * np.pi * plane_gap * plane_gap
    length_ref = 1
    for d in np.arange(length_ref, distance_gn, plane_gap):
        num_pts = 0
        num_blk_pts = 0
        num_sdw_pts = 0
        num_fs_pts = 0
        if(distance_gn - d < plane_gap):
            break
        else:
            c_x = n_x + d * xoxyz
            c_y = n_y + d * yoxyz
            c_z = n_z + d * zoxyz
            r_d = math.pow(d*(distance_gn - d)*(300/f)/distance_gn, 0.5)
            final_point_gap = point_gap*(d)/40
            gap_line_num = r_d // final_point_gap
            for i_iter in np.arange(-gap_line_num, gap_line_num+1, 1):
                for j_iter in np.arange(-gap_line_num, gap_line_num+1, 1):
                    i = i_iter
                    j = j_iter
                    if(i*i*final_point_gap*final_point_gap + j*j*final_point_gap*final_point_gap > r_d*r_d):
                        continue
                    else:
                        num_pts += 1
                        p_x = c_x + i * final_point_gap * yoxy - j * final_point_gap * zoxyz * yoxy
                        p_y = c_y - i * final_point_gap * xoxy - j * final_point_gap * zoxyz * yoxy
                        p_z = c_z + j * final_point_gap * xyoxyz
                        if(CheckPtGround(p_z)): 
                            num_blk_pts += 1
                            continue
                        else:
                            for t in trees:
                                if(t.CheckPtTrunk(p_x, p_y, p_z)):
                                    num_blk_pts += 1
                                elif(t.CheckPtFoliage(p_x, p_y, p_z)):
                                    num_sdw_pts += 1
            num_fs_pts = num_pts - num_blk_pts - num_sdw_pts
            sample_rec.append([num_pts, num_blk_pts, num_sdw_pts])     
    return sample_rec

# FAN with plane_gap == 0.3m

In [14]:
# this code is used to sample the number of points in the ellipsoid, here skipped for speed
# height_ellips = []
# for i in range(3):
#     height_ellips.append(SampPair(0, 0, (i*0.5+0.5), 100, 0, 0.5, 0, 0, 3.5, 2, 2.8, 3, 0.1, 4.88, 6.66, 3, 904.3, 0.25, 0.1))
# print(height_ellips)

For this pair,  0  trees are suspicious.
For this pair,  0  trees are suspicious.
For this pair,  0  trees are suspicious.
[[[165101, 4540, 0], [131705, 7933, 0], [109513, 9559, 0], [93617, 10408, 0], [81717, 10934, 0], [72457, 10959, 0], [65009, 10860, 0], [58957, 10585, 0], [53893, 10309, 0], [49629, 9996, 0], [45965, 9619, 0], [42777, 9355, 0], [39997, 9199, 0], [37577, 8736, 0], [35405, 8538, 0], [33437, 8199, 0], [31693, 8121, 0], [30089, 7707, 0], [28649, 7529, 0], [27337, 7381, 0], [26101, 7080, 0], [25033, 7012, 0], [24017, 6783, 0], [23045, 6572, 0], [22129, 6375, 0], [21353, 6227, 0], [20541, 6064, 0], [19861, 5946, 0], [19189, 5828, 0], [18513, 5560, 0], [17937, 5472, 0], [17401, 5402, 0], [16881, 5193, 0], [16365, 5125, 0], [15877, 4928, 0], [15445, 4888, 0], [15021, 4850, 0], [14617, 4685, 0], [14225, 4526, 0], [13849, 4498, 0], [13509, 4359, 0], [13157, 4343, 0], [12825, 4208, 0], [12541, 4088, 0], [12241, 4084, 0], [11945, 3961, 0], [11661, 3965, 0], [11425, 3863, 0], [1

In [8]:
# the result of the sampling
height_ellips = [[[165101, 4540, 0], [131705, 7933, 0], [109513, 9559, 0], [93617, 10408, 0], [81717, 10934, 0], [72457, 10959, 0], [65009, 10860, 0], [58957, 10585, 0], [53893, 10309, 0], [49629, 9996, 0], [45965, 9619, 0], [42777, 9355, 0], [39997, 9199, 0], [37577, 8736, 0], [35405, 8538, 0], [33437, 8199, 0], [31693, 8121, 0], [30089, 7707, 0], [28649, 7529, 0], [27337, 7381, 0], [26101, 7080, 0], [25033, 7012, 0], [24017, 6783, 0], [23045, 6572, 0], [22129, 6375, 0], [21353, 6227, 0], [20541, 6064, 0], [19861, 5946, 0], [19189, 5828, 0], [18513, 5560, 0], [17937, 5472, 0], [17401, 5402, 0], [16881, 5193, 0], [16365, 5125, 0], [15877, 4928, 0], [15445, 4888, 0], [15021, 4850, 0], [14617, 4685, 0], [14225, 4526, 0], [13849, 4498, 0], [13509, 4359, 0], [13157, 4343, 0], [12825, 4208, 0], [12541, 4088, 0], [12241, 4084, 0], [11945, 3961, 0], [11661, 3965, 0], [11425, 3863, 0], [11165, 3756, 0], [10909, 3651, 0], [10717, 3682, 0], [10453, 3573, 0], [10245, 3483, 0], [10037, 3396, 0], [9809, 3412, 0], [9649, 3342, 0], [9449, 3261, 0], [9265, 3181, 0], [9085, 3211, 0], [8937, 3147, 0], [8741, 3064, 0], [8585, 2996, 0], [8421, 2929, 0], [8293, 2972, 0], [8109, 2897, 0], [8005, 2851, 0], [7861, 2791, 0], [7721, 2730, 0], [7597, 2676, 0], [7449, 2710, 0], [7353, 2668, 0], [7209, 2609, 0], [7129, 2573, 0], [7009, 2519, 0], [6869, 2462, 0], [6777, 2420, 0], [6665, 2463, 0], [6573, 2424, 0], [6485, 2386, 0], [6369, 2336, 0], [6269, 2293, 0], [6181, 2253, 0], [6093, 2217, 0], [5997, 2176, 0], [5909, 2136, 0], [5829, 2187, 0], [5753, 2154, 0], [5673, 2118, 0], [5589, 2082, 0], [5497, 2043, 0], [5417, 2007, 0], [5369, 1987, 0], [5249, 1936, 0], [5209, 1918, 0], [5137, 1886, 0], [5065, 1854, 0], [4997, 1906, 0], [4941, 1878, 0], [4873, 1848, 0], [4825, 1828, 0], [4725, 1785, 0], [4693, 1771, 0], [4637, 1745, 0], [4577, 1719, 0], [4501, 1688, 0], [4461, 1670, 0], [4397, 1640, 0], [4349, 1620, 0], [4281, 1593, 0], [4237, 1571, 0], [4189, 1551, 0], [4149, 1604, 0], [4061, 1566, 0], [4025, 1551, 0], [3993, 1535, 0], [3945, 1513, 0], [3893, 1491, 0], [3833, 1468, 0], [3793, 1448, 0], [3761, 1432, 0], [3713, 1410, 0], [3673, 1394, 0], [3613, 1371, 0], [3577, 1353, 0], [3545, 1337, 0], [3513, 1323, 0], [3457, 1299, 0], [3409, 1281, 0], [3389, 1272, 0], [3341, 1248, 0], [3313, 1299, 0], [3281, 1285, 0], [3241, 1269, 0], [3197, 1252, 0], [3149, 1228, 0], [3125, 1216, 0], [3109, 1210, 0], [3061, 1188, 0], [3025, 1174, 0], [2989, 1159, 0], [2965, 1147, 0], [2941, 1135, 0], [2893, 1113, 0], [2869, 1103, 0], [2837, 1091, 0], [2801, 1076, 0], [2785, 1068, 0], [2749, 1050, 0], [2733, 1042, 0], [2701, 1030, 0], [2661, 1012, 0], [2617, 995, 0], [2601, 987, 0], [2593, 983, 0], [2561, 967, 0], [2537, 957, 0], [2501, 996, 0], [2469, 984, 0], [2449, 977, 0], [2417, 961, 0], [2393, 949, 0], [2377, 941, 0], [2361, 933, 0], [2337, 923, 0], [2313, 913, 0], [2285, 904, 0], [2241, 882, 0], [2233, 878, 0], [2217, 870, 0], [2185, 854, 0], [2177, 852, 0], [2145, 838, 0], [2109, 825, 0], [2093, 817, 0], [2085, 813, 0], [2061, 801, 0], [2029, 785, 0], [2025, 783, 0], [2001, 773, 0], [1993, 771, 0], [1941, 750, 0], [1933, 746, 0], [1917, 738, 0], [1893, 726, 0], [1885, 722, 0], [1869, 714, 0], [1861, 712, 0], [1829, 698, 0], [1789, 683, 0], [1789, 683, 0], [1765, 671, 0], [1757, 667, 0], [1741, 659, 0], [1725, 651, 0], [1701, 641, 0], [1685, 635, 0], [1669, 674, 0], [1649, 667, 0], [1633, 659, 0], [1617, 651, 0], [1605, 645, 0], [1581, 633, 0], [1565, 625, 0], [1565, 625, 0], [1541, 615, 0], [1533, 613, 0], [1489, 594, 0], [1481, 590, 0], [1481, 590, 0], [1473, 586, 0], [1457, 578, 0], [1433, 566, 0], [1433, 566, 0], [1405, 554, 0], [1389, 548, 0], [1369, 541, 0], [1361, 537, 0], [1353, 533, 0], [1353, 533, 0], [1321, 517, 0], [1313, 513, 0], [1305, 509, 0], [1281, 499, 0], [1273, 495, 0], [1257, 491, 0], [1237, 482, 0], [1229, 478, 0], [1217, 472, 0], [1201, 464, 0], [1201, 464, 0], [1185, 456, 0], [1177, 452, 0], [1153, 442, 0], [1153, 442, 0], [1129, 434, 0], [1117, 429, 0], [1109, 425, 0], [1101, 421, 0], [1093, 417, 0], [1085, 413, 0], [1069, 405, 0], [1049, 395, 0], [1041, 393, 0], [1041, 393, 0], [1033, 391, 0], [1005, 380, 0], [997, 376, 0], [989, 372, 0], [973, 364, 0], [973, 364, 0], [965, 360, 0], [949, 352, 0], [949, 352, 0], [933, 346, 0], [925, 344, 0], [889, 329, 0], [885, 327, 0], [885, 327, 0], [877, 323, 0], [869, 319, 0], [861, 315, 0], [853, 311, 0], [845, 307, 0], [829, 332, 0], [829, 332, 0], [805, 322, 0], [793, 319, 0], [793, 319, 0], [777, 311, 0], [777, 311, 0], [769, 307, 0], [757, 301, 0], [749, 297, 0], [749, 297, 0], [733, 289, 0], [725, 285, 0], [717, 283, 0], [697, 276, 0], [697, 276, 0], [681, 268, 0], [673, 264, 0], [673, 264, 0], [665, 260, 0], [665, 260, 0], [657, 256, 0], [641, 248, 0], [633, 244, 0], [621, 240, 0], [613, 238, 0], [601, 233, 0], [593, 229, 0], [593, 229, 0], [593, 229, 0], [577, 221, 0], [577, 221, 0], [561, 213, 0], [553, 209, 0], [553, 209, 0], [545, 207, 0], [529, 201, 0], [517, 196, 0], [517, 196, 0], [509, 192, 0], [505, 190, 0], [497, 186, 0], [489, 182, 0], [489, 182, 0], [481, 178, 0], [481, 178, 0], [465, 172, 0], [457, 168, 0], [437, 161, 0], [437, 161, 0], [437, 161, 0], [429, 157, 0], [421, 153, 0], [421, 153, 0], [421, 153, 0], [405, 145, 0], [401, 143, 0], [385, 137, 0], [385, 137, 0], [373, 134, 0], [373, 134, 0], [365, 130, 0], [357, 126, 0], [349, 122, 0], [349, 122, 0], [341, 118, 0], [341, 118, 0], [333, 114, 0], [325, 112, 0], [325, 112, 0], [305, 105, 0], [301, 103, 0], [293, 99, 0], [293, 99, 0], [293, 99, 0], [293, 99, 0], [277, 91, 0], [277, 91, 0], [261, 85, 0], [261, 85, 0], [253, 83, 0], [241, 78, 0], [241, 78, 0], [241, 78, 0], [241, 78, 0], [225, 70, 0], [221, 68, 0], [221, 68, 0], [213, 66, 0], [213, 66, 0], [193, 59, 0], [193, 59, 0], [185, 55, 0], [185, 55, 0], [177, 51, 0], [177, 51, 0], [177, 51, 0], [169, 49, 0], [161, 45, 0], [149, 41, 0], [145, 40, 0], [145, 40, 0], [137, 36, 0], [137, 36, 0], [137, 36, 0], [121, 30, 0], [121, 30, 0], [113, 28, 0], [109, 27, 0], [101, 23, 0], [101, 23, 0], [97, 21, 0], [97, 21, 0], [89, 19, 0], [89, 19, 0], [81, 17, 0], [69, 12, 0], [69, 12, 0], [69, 12, 0], [61, 10, 0], [57, 8, 0], [49, 6, 0], [45, 5, 0], [45, 5, 0], [37, 3, 0], [37, 3, 0], [29, 1, 0], [25, 0, 0], [21, 0, 0], [21, 0, 0], [13, 0, 0], [9, 0, 0], [5, 0, 0]], [[165101, 3877, 0], [131705, 7174, 0], [109513, 9043, 0], [93617, 9896, 0], [81717, 10181, 0], [72457, 10467, 0], [65009, 10147, 0], [58957, 10123, 0], [53893, 9861, 0], [49629, 9560, 0], [45965, 9195, 0], [42777, 8943, 0], [39997, 8594, 0], [37577, 8344, 0], [35405, 8154, 0], [33437, 7825, 0], [31693, 7570, 0], [30089, 7349, 0], [28649, 7177, 0], [27337, 7035, 0], [26101, 6742, 0], [25033, 6515, 0], [24017, 6457, 0], [23045, 6252, 0], [22129, 6061, 0], [21353, 5917, 0], [20541, 5760, 0], [19861, 5646, 0], [19189, 5385, 0], [18513, 5270, 0], [17937, 5186, 0], [17401, 5120, 0], [16881, 4915, 0], [16365, 4851, 0], [15877, 4658, 0], [15445, 4622, 0], [15021, 4455, 0], [14617, 4425, 0], [14225, 4270, 0], [13849, 4244, 0], [13509, 4109, 0], [13157, 4095, 0], [12825, 3962, 0], [12541, 3846, 0], [12241, 3844, 0], [11945, 3725, 0], [11661, 3614, 0], [11425, 3631, 0], [11165, 3526, 0], [10909, 3425, 0], [10717, 3456, 0], [10453, 3351, 0], [10245, 3263, 0], [10037, 3178, 0], [9809, 3196, 0], [9649, 3128, 0], [9449, 3049, 0], [9265, 2971, 0], [9085, 3003, 0], [8937, 2941, 0], [8741, 2860, 0], [8585, 2794, 0], [8421, 2729, 0], [8293, 2774, 0], [8109, 2699, 0], [8005, 2655, 0], [7861, 2597, 0], [7721, 2538, 0], [7597, 2486, 0], [7449, 2520, 0], [7353, 2480, 0], [7209, 2423, 0], [7129, 2387, 0], [7009, 2337, 0], [6869, 2280, 0], [6777, 2240, 0], [6665, 2196, 0], [6573, 2246, 0], [6485, 2208, 0], [6369, 2162, 0], [6269, 2119, 0], [6181, 2081, 0], [6093, 2047, 0], [5997, 2006, 0], [5909, 1968, 0], [5829, 2019, 0], [5753, 1988, 0], [5673, 1952, 0], [5589, 1918, 0], [5497, 1881, 0], [5417, 1845, 0], [5369, 1825, 0], [5249, 1778, 0], [5209, 1760, 0], [5137, 1728, 0], [5065, 1698, 0], [4997, 1673, 0], [4941, 1724, 0], [4873, 1694, 0], [4825, 1674, 0], [4725, 1633, 0], [4693, 1621, 0], [4637, 1595, 0], [4577, 1569, 0], [4501, 1540, 0], [4461, 1524, 0], [4397, 1494, 0], [4349, 1474, 0], [4281, 1449, 0], [4237, 1429, 0], [4189, 1409, 0], [4149, 1462, 0], [4061, 1424, 0], [4025, 1411, 0], [3993, 1397, 0], [3945, 1375, 0], [3893, 1353, 0], [3833, 1330, 0], [3793, 1312, 0], [3761, 1298, 0], [3713, 1276, 0], [3673, 1260, 0], [3613, 1237, 0], [3577, 1221, 0], [3545, 1207, 0], [3513, 1193, 0], [3457, 1169, 0], [3409, 1151, 0], [3389, 1144, 0], [3341, 1122, 0], [3313, 1173, 0], [3281, 1159, 0], [3241, 1143, 0], [3197, 1126, 0], [3149, 1104, 0], [3125, 1094, 0], [3109, 1088, 0], [3061, 1066, 0], [3025, 1052, 0], [2989, 1037, 0], [2965, 1027, 0], [2941, 1017, 0], [2893, 995, 0], [2869, 985, 0], [2837, 973, 0], [2801, 958, 0], [2785, 952, 0], [2749, 936, 0], [2733, 928, 0], [2701, 916, 0], [2661, 898, 0], [2617, 881, 0], [2601, 875, 0], [2593, 871, 0], [2561, 857, 0], [2537, 847, 0], [2501, 831, 0], [2469, 874, 0], [2449, 867, 0], [2417, 851, 0], [2393, 841, 0], [2377, 833, 0], [2361, 827, 0], [2337, 817, 0], [2313, 807, 0], [2285, 798, 0], [2241, 776, 0], [2233, 774, 0], [2217, 766, 0], [2185, 752, 0], [2177, 750, 0], [2145, 736, 0], [2109, 723, 0], [2093, 715, 0], [2085, 711, 0], [2061, 701, 0], [2029, 687, 0], [2025, 685, 0], [2001, 675, 0], [1993, 673, 0], [1941, 652, 0], [1933, 648, 0], [1917, 642, 0], [1893, 630, 0], [1885, 628, 0], [1869, 620, 0], [1861, 618, 0], [1829, 604, 0], [1789, 589, 0], [1789, 589, 0], [1765, 579, 0], [1757, 575, 0], [1741, 569, 0], [1725, 561, 0], [1701, 551, 0], [1685, 545, 0], [1669, 539, 0], [1649, 577, 0], [1633, 569, 0], [1617, 561, 0], [1605, 555, 0], [1581, 545, 0], [1565, 539, 0], [1565, 539, 0], [1541, 529, 0], [1533, 527, 0], [1489, 508, 0], [1481, 504, 0], [1481, 504, 0], [1473, 500, 0], [1457, 494, 0], [1433, 484, 0], [1433, 484, 0], [1405, 472, 0], [1389, 466, 0], [1369, 459, 0], [1361, 455, 0], [1353, 451, 0], [1353, 451, 0], [1321, 437, 0], [1313, 433, 0], [1305, 431, 0], [1281, 421, 0], [1273, 417, 0], [1257, 413, 0], [1237, 404, 0], [1229, 400, 0], [1217, 394, 0], [1201, 388, 0], [1201, 388, 0], [1185, 382, 0], [1177, 378, 0], [1153, 368, 0], [1153, 368, 0], [1129, 360, 0], [1117, 355, 0], [1109, 351, 0], [1101, 347, 0], [1093, 345, 0], [1085, 341, 0], [1069, 335, 0], [1049, 325, 0], [1041, 323, 0], [1041, 323, 0], [1033, 321, 0], [1005, 310, 0], [997, 306, 0], [989, 302, 0], [973, 296, 0], [973, 296, 0], [965, 292, 0], [949, 286, 0], [949, 286, 0], [933, 280, 0], [925, 278, 0], [889, 263, 0], [885, 261, 0], [885, 261, 0], [877, 259, 0], [869, 255, 0], [861, 251, 0], [853, 249, 0], [845, 245, 0], [829, 270, 0], [829, 270, 0], [805, 260, 0], [793, 257, 0], [793, 257, 0], [777, 249, 0], [777, 249, 0], [769, 245, 0], [757, 239, 0], [749, 237, 0], [749, 237, 0], [733, 231, 0], [725, 227, 0], [717, 225, 0], [697, 218, 0], [697, 218, 0], [681, 210, 0], [673, 206, 0], [673, 206, 0], [665, 204, 0], [665, 204, 0], [657, 200, 0], [641, 194, 0], [633, 190, 0], [621, 186, 0], [613, 184, 0], [601, 179, 0], [593, 175, 0], [593, 175, 0], [593, 175, 0], [577, 169, 0], [577, 169, 0], [561, 161, 0], [553, 159, 0], [553, 159, 0], [545, 157, 0], [529, 151, 0], [517, 146, 0], [517, 146, 0], [509, 142, 0], [505, 140, 0], [497, 138, 0], [489, 134, 0], [489, 134, 0], [481, 132, 0], [481, 132, 0], [465, 126, 0], [457, 122, 0], [437, 115, 0], [437, 115, 0], [437, 115, 0], [429, 113, 0], [421, 109, 0], [421, 109, 0], [421, 109, 0], [405, 103, 0], [401, 101, 0], [385, 95, 0], [385, 95, 0], [373, 92, 0], [373, 92, 0], [365, 88, 0], [357, 86, 0], [349, 82, 0], [349, 82, 0], [341, 80, 0], [341, 80, 0], [333, 76, 0], [325, 74, 0], [325, 74, 0], [305, 67, 0], [301, 65, 0], [293, 63, 0], [293, 63, 0], [293, 63, 0], [293, 63, 0], [277, 57, 0], [277, 57, 0], [261, 51, 0], [261, 51, 0], [253, 49, 0], [241, 46, 0], [241, 46, 0], [241, 46, 0], [241, 46, 0], [225, 40, 0], [221, 38, 0], [221, 38, 0], [213, 36, 0], [213, 36, 0], [193, 31, 0], [193, 31, 0], [185, 27, 0], [185, 27, 0], [177, 25, 0], [177, 25, 0], [177, 25, 0], [169, 23, 0], [161, 21, 0], [149, 17, 0], [145, 16, 0], [145, 16, 0], [137, 14, 0], [137, 14, 0], [137, 14, 0], [121, 10, 0], [121, 10, 0], [113, 8, 0], [109, 7, 0], [101, 5, 0], [101, 5, 0], [97, 5, 0], [97, 5, 0], [89, 3, 0], [89, 3, 0], [81, 1, 0], [69, 0, 0], [69, 0, 0], [69, 0, 0], [61, 0, 0], [57, 0, 0], [49, 0, 0], [45, 0, 0], [45, 0, 0], [37, 0, 0], [37, 0, 0], [29, 0, 0], [25, 0, 0], [21, 0, 0], [21, 0, 0], [13, 0, 0], [9, 0, 0], [5, 0, 0]], [[165101, 3455, 0], [131705, 6682, 0], [109513, 8535, 0], [93617, 9390, 0], [81717, 9685, 0], [72457, 9983, 0], [65009, 9679, 0], [58957, 9667, 0], [53893, 9417, 0], [49629, 9128, 0], [45965, 8775, 0], [42777, 8535, 0], [39997, 8196, 0], [37577, 7956, 0], [35405, 7774, 0], [33437, 7455, 0], [31693, 7208, 0], [30089, 6995, 0], [28649, 6829, 0], [27337, 6693, 0], [26101, 6408, 0], [25033, 6187, 0], [24017, 6135, 0], [23045, 5934, 0], [22129, 5749, 0], [21353, 5611, 0], [20541, 5458, 0], [19861, 5348, 0], [19189, 5095, 0], [18513, 4984, 0], [17937, 4904, 0], [17401, 4840, 0], [16881, 4641, 0], [16365, 4579, 0], [15877, 4392, 0], [15445, 4358, 0], [15021, 4195, 0], [14617, 4167, 0], [14225, 4016, 0], [13849, 3994, 0], [13509, 3861, 0], [13157, 3849, 0], [12825, 3720, 0], [12541, 3608, 0], [12241, 3606, 0], [11945, 3491, 0], [11661, 3384, 0], [11425, 3401, 0], [11165, 3300, 0], [10909, 3201, 0], [10717, 3234, 0], [10453, 3131, 0], [10245, 3045, 0], [10037, 2964, 0], [9809, 2982, 0], [9649, 2916, 0], [9449, 2839, 0], [9265, 2765, 0], [9085, 2797, 0], [8937, 2737, 0], [8741, 2658, 0], [8585, 2594, 0], [8421, 2531, 0], [8293, 2576, 0], [8109, 2505, 0], [8005, 2461, 0], [7861, 2405, 0], [7721, 2348, 0], [7597, 2298, 0], [7449, 2334, 0], [7353, 2294, 0], [7209, 2239, 0], [7129, 2205, 0], [7009, 2155, 0], [6869, 2102, 0], [6777, 2062, 0], [6665, 2020, 0], [6573, 2070, 0], [6485, 2034, 0], [6369, 1988, 0], [6269, 1947, 0], [6181, 1911, 0], [6093, 1877, 0], [5997, 1838, 0], [5909, 1802, 0], [5829, 1853, 0], [5753, 1822, 0], [5673, 1788, 0], [5589, 1756, 0], [5497, 1719, 0], [5417, 1685, 0], [5369, 1667, 0], [5249, 1620, 0], [5209, 1604, 0], [5137, 1574, 0], [5065, 1544, 0], [4997, 1519, 0], [4941, 1570, 0], [4873, 1542, 0], [4825, 1524, 0], [4725, 1483, 0], [4693, 1471, 0], [4637, 1447, 0], [4577, 1423, 0], [4501, 1394, 0], [4461, 1378, 0], [4397, 1350, 0], [4349, 1332, 0], [4281, 1307, 0], [4237, 1287, 0], [4189, 1269, 0], [4149, 1322, 0], [4061, 1286, 0], [4025, 1273, 0], [3993, 1259, 0], [3945, 1239, 0], [3893, 1217, 0], [3833, 1196, 0], [3793, 1178, 0], [3761, 1164, 0], [3713, 1144, 0], [3673, 1130, 0], [3613, 1107, 0], [3577, 1091, 0], [3545, 1077, 0], [3513, 1065, 0], [3457, 1043, 0], [3409, 1025, 0], [3389, 1018, 0], [3341, 996, 0], [3313, 1047, 0], [3281, 1035, 0], [3241, 1019, 0], [3197, 1004, 0], [3149, 982, 0], [3125, 972, 0], [3109, 966, 0], [3061, 946, 0], [3025, 932, 0], [2989, 919, 0], [2965, 909, 0], [2941, 899, 0], [2893, 877, 0], [2869, 869, 0], [2837, 859, 0], [2801, 844, 0], [2785, 838, 0], [2749, 822, 0], [2733, 814, 0], [2701, 804, 0], [2661, 788, 0], [2617, 771, 0], [2601, 765, 0], [2593, 761, 0], [2561, 747, 0], [2537, 739, 0], [2501, 723, 0], [2469, 766, 0], [2449, 759, 0], [2417, 745, 0], [2393, 735, 0], [2377, 727, 0], [2361, 721, 0], [2337, 711, 0], [2313, 703, 0], [2285, 694, 0], [2241, 674, 0], [2233, 672, 0], [2217, 664, 0], [2185, 650, 0], [2177, 648, 0], [2145, 636, 0], [2109, 623, 0], [2093, 617, 0], [2085, 613, 0], [2061, 603, 0], [2029, 589, 0], [2025, 587, 0], [2001, 579, 0], [1993, 577, 0], [1941, 558, 0], [1933, 554, 0], [1917, 548, 0], [1893, 536, 0], [1885, 534, 0], [1869, 528, 0], [1861, 526, 0], [1829, 512, 0], [1789, 499, 0], [1789, 499, 0], [1765, 489, 0], [1757, 485, 0], [1741, 479, 0], [1725, 473, 0], [1701, 463, 0], [1685, 457, 0], [1669, 453, 0], [1649, 489, 0], [1633, 481, 0], [1617, 475, 0], [1605, 469, 0], [1581, 459, 0], [1565, 453, 0], [1565, 453, 0], [1541, 445, 0], [1533, 443, 0], [1489, 424, 0], [1481, 422, 0], [1481, 422, 0], [1473, 418, 0], [1457, 412, 0], [1433, 402, 0], [1433, 402, 0], [1405, 392, 0], [1389, 386, 0], [1369, 379, 0], [1361, 377, 0], [1353, 373, 0], [1353, 373, 0], [1321, 359, 0], [1313, 355, 0], [1305, 353, 0], [1281, 345, 0], [1273, 341, 0], [1257, 337, 0], [1237, 330, 0], [1229, 326, 0], [1217, 320, 0], [1201, 314, 0], [1201, 314, 0], [1185, 308, 0], [1177, 306, 0], [1153, 296, 0], [1153, 296, 0], [1129, 288, 0], [1117, 285, 0], [1109, 281, 0], [1101, 277, 0], [1093, 275, 0], [1085, 271, 0], [1069, 265, 0], [1049, 257, 0], [1041, 255, 0], [1041, 255, 0], [1033, 253, 0], [1005, 244, 0], [997, 240, 0], [989, 236, 0], [973, 230, 0], [973, 230, 0], [965, 226, 0], [949, 222, 0], [949, 222, 0], [933, 216, 0], [925, 214, 0], [889, 201, 0], [885, 199, 0], [885, 199, 0], [877, 197, 0], [869, 193, 0], [861, 191, 0], [853, 189, 0], [845, 185, 0], [829, 208, 0], [829, 208, 0], [805, 200, 0], [793, 197, 0], [793, 197, 0], [777, 191, 0], [777, 191, 0], [769, 187, 0], [757, 181, 0], [749, 179, 0], [749, 179, 0], [733, 173, 0], [725, 171, 0], [717, 169, 0], [697, 162, 0], [697, 162, 0], [681, 156, 0], [673, 152, 0], [673, 152, 0], [665, 150, 0], [665, 150, 0], [657, 146, 0], [641, 142, 0], [633, 138, 0], [621, 134, 0], [613, 132, 0], [601, 129, 0], [593, 125, 0], [593, 125, 0], [593, 125, 0], [577, 119, 0], [577, 119, 0], [561, 113, 0], [553, 111, 0], [553, 111, 0], [545, 109, 0], [529, 103, 0], [517, 100, 0], [517, 100, 0], [509, 96, 0], [505, 94, 0], [497, 92, 0], [489, 90, 0], [489, 90, 0], [481, 88, 0], [481, 88, 0], [465, 82, 0], [457, 80, 0], [437, 73, 0], [437, 73, 0], [437, 73, 0], [429, 71, 0], [421, 69, 0], [421, 69, 0], [421, 69, 0], [405, 63, 0], [401, 61, 0], [385, 57, 0], [385, 57, 0], [373, 54, 0], [373, 54, 0], [365, 52, 0], [357, 50, 0], [349, 46, 0], [349, 46, 0], [341, 44, 0], [341, 44, 0], [333, 42, 0], [325, 40, 0], [325, 40, 0], [305, 35, 0], [301, 33, 0], [293, 31, 0], [293, 31, 0], [293, 31, 0], [293, 31, 0], [277, 27, 0], [277, 27, 0], [261, 23, 0], [261, 23, 0], [253, 21, 0], [241, 18, 0], [241, 18, 0], [241, 18, 0], [241, 18, 0], [225, 14, 0], [221, 14, 0], [221, 14, 0], [213, 12, 0], [213, 12, 0], [193, 7, 0], [193, 7, 0], [185, 7, 0], [185, 7, 0], [177, 5, 0], [177, 5, 0], [177, 5, 0], [169, 3, 0], [161, 3, 0], [149, 1, 0], [145, 0, 0], [145, 0, 0], [137, 0, 0], [137, 0, 0], [137, 0, 0], [121, 0, 0], [121, 0, 0], [113, 0, 0], [109, 0, 0], [101, 0, 0], [101, 0, 0], [97, 0, 0], [97, 0, 0], [89, 0, 0], [89, 0, 0], [81, 0, 0], [69, 0, 0], [69, 0, 0], [69, 0, 0], [61, 0, 0], [57, 0, 0], [49, 0, 0], [45, 0, 0], [45, 0, 0], [37, 0, 0], [37, 0, 0], [29, 0, 0], [25, 0, 0], [21, 0, 0], [21, 0, 0], [13, 0, 0], [9, 0, 0], [5, 0, 0]]]

In [9]:
# print(np.shape(height_ellips))
h05 = height_ellips[0]
h10 = height_ellips[1]
h15 = height_ellips[2]

num_pts_arr_ellips = 0
num_blk_arr_ellips = 0
num_sdw_arr_ellips = 0
for plane in h05:
    num_pts_arr_ellips += plane[0]
    num_blk_arr_ellips += plane[1]
    num_sdw_arr_ellips += plane[2]
blk_rate = num_blk_arr_ellips/num_pts_arr_ellips
sdw_rate = num_sdw_arr_ellips/num_pts_arr_ellips
print(blk_rate, sdw_rate, 1 - blk_rate - sdw_rate)

num_pts_arr_ellips = 0
num_blk_arr_ellips = 0
num_sdw_arr_ellips = 0
for plane in h10:
    num_pts_arr_ellips += plane[0]
    num_blk_arr_ellips += plane[1]
    num_sdw_arr_ellips += plane[2]
blk_rate = num_blk_arr_ellips/num_pts_arr_ellips
sdw_rate = num_sdw_arr_ellips/num_pts_arr_ellips
print(blk_rate, sdw_rate, 1 - blk_rate - sdw_rate)

num_pts_arr_ellips = 0
num_blk_arr_ellips = 0
num_sdw_arr_ellips = 0
for plane in h15:
    num_pts_arr_ellips += plane[0]
    num_blk_arr_ellips += plane[1]
    num_sdw_arr_ellips += plane[2]
blk_rate = num_blk_arr_ellips/num_pts_arr_ellips
sdw_rate = num_sdw_arr_ellips/num_pts_arr_ellips
print(blk_rate, sdw_rate, 1 - blk_rate - sdw_rate)


0.24713133396974687 0.0 0.7528686660302532
0.22736260515238688 0.0 0.7726373948476131
0.20898376184465958 0.0 0.7910162381553404


In [11]:
# Do least-mean-square fitting for ffz_whole:
def ffz_whole(x, i, j, d):
#     length_ref = np.full(len(d), 0.3)
    ESP_log = []
    ellips = []
    num_pts_arr = []
    num_blk_arr = []
    num_sdw_arr = []
    blk_rate_arr = []
    sdw_rate_arr = []
    
    for item in range(len(i)):
#         print(int(i[item]))
        ellips.append(height_ellips[int(i[item])])
    for item in range(len(d)):
        num_pts_arr_ellips = 0
        num_blk_arr_ellips = 0
        num_sdw_arr_ellips = 0
        for plane in ellips[item]:
            num_pts_arr_ellips += plane[0]
            num_blk_arr_ellips += plane[1]
            num_sdw_arr_ellips += plane[2]
        blk_rate_arr.append(num_blk_arr_ellips/num_pts_arr_ellips)
        sdw_rate_arr.append(num_sdw_arr_ellips/num_pts_arr_ellips)
    for item in range(len(d)):
        log_rate = 2*(1 - blk_rate_arr[item]) + x[0]*blk_rate_arr[item] + 0*sdw_rate_arr[item]
        pl_log_single = 78.59 + 10 * log_rate * np.log10(d[item])
        ESP_log.append(22 - pl_log_single)
    return ESP_log

def fun(x, i, j, d, y):
#     print(np.array(ffz_whole(x, i, j, d)) - np.array(y))
    return np.array(ffz_whole(x, i, j, d)) - np.array(y)

x0 = np.array([10])
# print(len(testh["i"].tolist()))
res = least_squares(fun, x0, args=(testh["i"].tolist(), testh["j"].tolist(), testh["d"].tolist(), testh["ESP"].tolist()), verbose=1)
print(res.x)
# print(abs(res.fun))
print(np.mean(abs(res.fun)))

Both `ftol` and `xtol` termination conditions are satisfied.
Function evaluations 3, initial cost 8.5053e+04, final cost 7.4574e+03, first-order optimality 1.67e-04.
[5.38812416]
5.466976455552731
